In [6]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_lfw_people
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Load the LFW dataset
lfw_people = fetch_lfw_people(min_faces_per_person=10, resize=0.5)
X = lfw_people.data
y = lfw_people.target
target_names = lfw_people.target_names

# Check the original shape of images
image_shape = lfw_people.images.shape[1:]
h, w = image_shape  # Update the height and width based on the actual shape

# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Extract histograms
def extract_histograms(X):
    histograms = []
    for image in X:
        image_reshaped = image.reshape(62, 47, 3)  # Use the correct shape
        hist_r, _ = np.histogram(image_reshaped[:, :, 0], bins=256, range=(0, 255))
        hist_g, _ = np.histogram(image_reshaped[:, :, 1], bins=256, range=(0, 255))
        hist_b, _ = np.histogram(image_reshaped[:, :, 2], bins=256, range=(0, 255))
        histograms.append(np.concatenate((hist_r, hist_g, hist_b)))
    return np.array(histograms)

X_histograms = extract_histograms(X_scaled)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_histograms, y, test_size=0.2, random_state=42)

# Train the SVM classifier
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)

# Make predictions
y_pred = svm.predict(X_test)

# Calculate accuracy and classification report
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred, target_names=target_names))

# Visualize images
def plot_gallery(images, titles, h, w, n_row=3, n_col=4):
    plt.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)

    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(images[i].reshape(h, w, 3), cmap=plt.cm.gray)
        plt.title(titles[i], size=12)
        plt.xticks(())
        plt.yticks(())

# Prepare titles for the predicted and true labels
predicted_titles = [f'pred: {target_names[y_pred[i]]}\ntrue: {target_names[y_test[i]]}' for i in range(y_test.shape[0])]

# Plot the gallery of images
plot_gallery(X_test, predicted_titles, h, w)
plt.show()

ValueError: cannot reshape array of size 2914 into shape (62,47,3)